# Homework 2: Decipherment

### Group **northernwolfpack**

* Chithra Bhat, cbhat
* Gustavo Felhberg, gfelhber
* Heikal Badrulhisham, hbadrulh

# Baseline Model

The baseline mode, using an ngram model and a beam search, was implemented by translating the pseudocode in the homework description into Python code.

# ngram Model (best results)

This is an extension of the baseline method using a beam search and an ngram language model. The additions are:

-Ordering the list of English alphabet letters by letter frequency. Initially the English alphabet referred to by the beam search was in the traditional order ('abcd...').

-Obtaining the extension order of cipher symbols iteratively during the beam search, rather than obtaining a complete order prior. At each iteration of the beam search, the next cipher symbol for extension is chosen such that the next hypothesis expansion would result in a decipherment with the maximal contiguous ngrams.


### Ideas tested
The following are additions to the main algorithm that were implemented and tested, but not kept because they led to performance deterioration:

* **constraints based on English orthography**: 
1) limits on the length of consecutive consonants or vowels. 2) insisting that any consonant or vowel sequences in a potential decipherment must also occur in the Wikipedia corpus. These constraints were tested as outright ban or gradient penalties (more constraint violations lower the score for a hypothesis in the beam search)
* **variable extension limits**: Extension limits were changed from a fixed extension limit for all plaintext letters, to variable extension limits for each letter. More frequent English characters were given higher extension limits.

### Main files:

* **ngram_solution.ipynb**: working file for the ngram approach (not for final submission)

* **orthography.py**: file for deriving constraints on English orthography

* **ngram_solution_alternative.ipynb**: version of the model with a few differences in some methods, but with the same overall idea.

* **sandhill_cranes.txt**: 1:1 substitution cipher text we used to test our models. The 'cipher' is capitalization of the plain text.

# NLM Model

We followed the approach described in the Thesis "Decipherment of Substitution Ciphers with Neural Language Models". To implement the sampling, we used the method nlm.py/next_chars(). However, this process was taking too long, so we tried to reduce the sampling in a way that, instead of sampling the whole sentence for every new hypothesis, we sampled only the minimun lenght of the sentence that covers all the mapped characters for the current new hypothesis. 

**Example:** Assume that there are tree mapped hypotheses. The mapped sentence would be: ".......................a.............v........c..a..va.....c.....". Instead of sampling all the gaps for the whole sentence, we sampled until the first 'c' and scored the sentence up to the first character 'c'. Although it improved a little in terms of running time, we noted that the results lost some readability.

The auxiliary methods for this model are the same used for the ngram model.

After several trials with different combinations and considering the huge amount of time for each test (not using CPU), we decided to focus only in the Ngram solution.

### Main files:

* **nlm_solution.ipynb:** This file contains all the methods used to create the NLM model.



In [1]:
from collections import Counter
import sys
sys.path.append('../')
from ngram import *
from nltk.util import ngrams

# ngram model for scoring hypotheses
ngram_model = LM("data/6-gram-wiki-char.lm.bz2", n=6, verbose=False)

def read_file(filename):
    if filename[-4:] == ".bz2":
        with bz2.open(filename, 'rt') as f:
            content = f.read()
            f.close()
    else:
        with open(filename, 'r') as f:
            content = f.read()
            f.close()
    return content

Reading language model from data/6-gram-wiki-char.lm.bz2...
Done.


In [2]:
# Beam search algorithm
def beam_search(plaintext_alph, cipher_text, ext_order, ext_limits=1, beam_size=1):
    # partial hypotheses
    scored_hypotheses = [(0, [])]
    hypothesis_extensions = []

    for cipher_sym in ext_order:
        for hyp in scored_hypotheses:
            for pl_sym in plaintext_alph:
                new_hypothesis = hyp[1] + [(pl_sym, cipher_sym)]

                if within_ext_limits(ext_limits, new_hypothesis):
                    hypothesis_extensions.append((score(new_hypothesis, cipher_text), new_hypothesis))

            if hypothesis_extensions:
                hypothesis_extensions = histogram_prune(hypothesis_extensions, beam_size)
                scored_hypotheses = [h for h in hypothesis_extensions]

        hypothesis_extensions.clear()
    return winning_hypothesis(scored_hypotheses)


# If the hypothesis exceeds the constraint on many-to-one mapping
def within_ext_limits(limit, hyp):
    plaintxt_sym_counter = Counter([tup[0] for tup in hyp])
    return not any(plaintxt_sym_counter[k] > limit[k] for k in plaintxt_sym_counter)


# Hypothesis scoring function
def score(hypothesis, text):
    decipherment = ''.join([g_funct(hypothesis, ch) for ch in text])
    bitstring = ''.join(['o' if ch != '_' else '.' for ch in decipherment])
    return ngram_model.score_bitstring(decipherment, bitstring)


# Returns plaintext string for a cipher symbol given a hypothesized mapping
def g_funct(hypothesis, cipher_sym):
    for tup in hypothesis:
        if tup[1] == cipher_sym:
            return tup[0]
    return '_'


# Returns a sublist of n best scoring hypotheses
def histogram_prune(hypotheses, n=1):
    hypotheses.sort(reverse=True)
    return hypotheses[:n]


# Returns the best hypothesis
def winning_hypothesis(hypotheses):
    return histogram_prune(hypotheses, 1)[0][1]


# Get an extension order based on contiguous deciphered ngrams
def get_ext_order(alphabet, cipher):
    ext_order = [alphabet[0]]
    alphabet.pop(0)

    while alphabet:
        max_sum = weighted_sum(alphabet[0], cipher, ext_order)
        max_char = alphabet[0]

        for a in alphabet[1:]:
            curr_sum = weighted_sum(a, cipher, ext_order)

            if curr_sum > max_sum:
                max_sum = curr_sum
                max_char = a

        ext_order.append(max_char)
        alphabet.remove(max_char)

    return ext_order


# Calculate the weighted sum for ext order candidates
def weighted_sum(ch, cipher, ext_order):
    sum = 0
    for n in range(2, 7):
        grams = [g for g in ngrams(cipher, n) if all(c in ext_order + [ch] for c in g)]
        sum += len(grams) * n

    return sum

In [3]:
# Cipher and plaintext files
cipher = read_file("data/cipher.txt").replace('\n', '')
plaintxt = read_file("data/default.wiki.txt.bz2")

# Cipher and plaintext alphabets
cipher_count = Counter([ch for ch in cipher if not ch == '\n'])
# Order cipher letters to maximize contiguous ngrams for each partial hypothesis
ext_order = [tup[0] for tup in cipher_count.most_common()]
ext_order = get_ext_order(ext_order, cipher)
# English alphabet ordered by letter frequency
eng_alphabet = [ch for ch in 'etaoinshrdlcumwfgypbvkjxqz']

# Variable extension limits for each plaintext letter (can be modified to simulate constant ext_limits
# ext_limit = {'e': 4, 't': 3, 'a': 3, 'o': 3, 'i': 3, 'n': 2, 's': 2, 'h': 2,'r': 2, 'd': 2, 
#              'l': 2, 'c': 2, 'u': 2, 'm': 2, 'w': 1, 'f': 1, 'g': 1, 'y': 1, 'p': 1, 'b': 1, 
#              'v': 1, 'k': 1, 'j': 1, 'x': 1, 'q': 1, 'z': 1}

e = 6
ext_limit = {'e': e, 't': e, 'a': e, 'o': e, 'i': e, 'n': e, 's': e, 'h': e,'r': e, 'd': e, 
              'l': e, 'c': e, 'u': e, 'm': e, 'w': e, 'f': e, 'g': e, 'y': e, 'p': e, 'b': e,
              'v': e, 'k': e, 'j': e, 'x': e, 'q': e, 'z': e}

In [4]:
# Get the best decipherment hypothesis
best_hypothesis = beam_search(eng_alphabet, cipher, ext_order, ext_limit, 26)
# Decipher cipher text
decipherment = ''.join([g_funct(best_hypothesis, ch) for ch in cipher])
print(best_hypothesis)

[('r', '—'), ('e', 'W'), ('e', '∏'), ('a', 'X'), ('s', '≈'), ('e', '–'), ('r', 'V'), ('e', 'B'), ('s', '•'), ('e', '+'), ('h', '\\'), ('h', '∑'), ('t', 'ƒ'), ('a', '∞'), ('s', 'F'), ('t', 'H'), ('a', 'Ç'), ('t', 'º'), ('t', '“'), ('e', 'E'), ('s', 'I'), ('t', 'y'), ('i', 'G'), ('s', 'K'), ('t', '£'), ('i', 'æ'), ('s', 'π'), ('r', 'P'), ('o', 'D'), ('n', 'R'), ('i', 'T'), ('a', 'A'), ('n', 'µ'), ('a', 'O'), ('n', 'Q'), ('a', '∫'), ('o', 'M'), ('n', 'J'), ('i', 'À'), ('i', 'u'), ('n', '√'), ('u', 'Z'), ('o', '/'), ('c', 'L'), ('n', '‘'), ('y', '^'), ('o', 'S'), ('o', 'N'), ('d', 'Ã'), ('r', '¢'), ('b', '§'), ('r', 'Ω'), ('r', '∆'), ('i', 'j')]


In [5]:
print(decipherment)

tarouoieaianseaseeresitsattrssartsontyisirrinontontorotheranaiarndreiycountthestresitseriotsstarainbornonstedairieinohectorhesaretoiynninusresteraincottheroneisahorseitisunearesnenctdynatstoneitthtsosannarnoininactorestsitsaninticodoaturrbieiathereherinarysotdrisarobiaasooitoneotaaurarheresarereshinestateaainnineettreyorerusitseeitatahcnetaseatbaaothrcasresthhasnranansainusninreinseterneareattesaretotsisi
